In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d fabianavinci/guitar-chords-v3

In [ ]:
!unzip /content/guitar-chords-v3.zip

In [4]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize

In [ ]:
!mv /content/Train/* Chords/
!find /content/Test -mindepth 1 -maxdepth 1 -type d -exec mv {}/* /content/Chords/{} \;

In [12]:
data_dir = '/content/Chords'
classes = ['Am', 'Bb', 'Bdim', 'C', 'Dm', 'Em', 'F', 'G']

def data_preprocessing(data_dir, classes, target_shape=(128, 128)):
    data = [], labels = []
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in [f for f in os.listdir(class_dir) if f.endswith('.wav')]:
            file_path = os.path.join(class_dir, filename)
            audio_data, _ = librosa.load(file_path, sr=None)
            mel_spectrogram = resize(np.expand_dims(librosa.feature.melspectrogram(y=audio_data, sr=_), axis=-1), target_shape)
            data.append(mel_spectrogram)
            labels.append(i)
    return np.array(data), np.array(labels)

In [13]:
data, labels = data_preprocessing(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [14]:
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(96, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)


In [16]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/200
44/44 [==============================] - 10s 29ms/step - loss: 2.0987 - accuracy: 0.3473 - val_loss: 1.6873 - val_accuracy: 0.4290
Epoch 2/200
44/44 [==============================] - 1s 23ms/step - loss: 1.1390 - accuracy: 0.6072 - val_loss: 1.5793 - val_accuracy: 0.5312
Epoch 3/200
44/44 [==============================] - 1s 27ms/step - loss: 0.8244 - accuracy: 0.7152 - val_loss: 1.6440 - val_accuracy: 0.4886
Epoch 4/200
44/44 [==============================] - 1s 28ms/step - loss: 0.5603 - accuracy: 0.8111 - val_loss: 1.5834 - val_accuracy: 0.5000
Epoch 5/200
44/44 [==============================] - 1s 23ms/step - loss: 0.4043 - accuracy: 0.8778 - val_loss: 2.7544 - val_accuracy: 0.3665
Epoch 6/200
44/44 [==============================] - 1s 25ms/step - loss: 0.3182 - accuracy: 0.8956 - val_loss: 3.4332 - val_accuracy: 0.4006
Epoch 7/200
44/44 [==============================] - 1s 25ms/step - loss: 0.2556 - accuracy: 0.9197 - val_loss: 1.7231 - val_accuracy: 0.5540
Epoch

In [18]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.9488636255264282


In [40]:
#model.save('Chord_Classifier.keras')
model = load_model('Chord_Classifier.keras')
target_shape = (128, 128)
classes = ['Am', 'Bb', 'Bdim', 'C', 'Dm', 'Em', 'F', 'G']

def audio_test(file_path, model):
    audio_data, _ = librosa.load(file_path, sr=None)
    mel_spectrogram = resize(np.expand_dims(librosa.feature.melspectrogram(y=audio_data, sr=_), axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))
    predictions = model.predict(mel_spectrogram)
    class_probabilities = predictions[0]
    predicted_class_index = np.argmax(class_probabilities)
    return predicted_class_index

file_path = '/content/Chords/C/C_ClassicA_Miguel_3.wav'
predicted_class_index = audio_test(file_path, model)

predicted_class = classes[predicted_class_index]
print(f'This is {predicted_class} chord')

1/1 [==============================] - 0s 151ms/step
This is C chord
